In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(100))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [9]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('leaky-relu')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 29s 83ms/step - loss: 4.6928 - accuracy: 0.0262 - top3_acc: 0.0708 - top_k_categorical_accuracy: 0.1112 - val_loss: 4.0826 - val_accuracy: 0.1011 - val_top3_acc: 0.2290 - val_top_k_categorical_accuracy: 0.3065
Epoch 2/50
313/313 [==============================] - 25s 81ms/step - loss: 4.0764 - accuracy: 0.0755 - top3_acc: 0.1787 - top_k_categorical_accuracy: 0.2553 - val_loss: 3.8335 - val_accuracy: 0.1360 - val_top3_acc: 0.2787 - val_top_k_categorical_accuracy: 0.3698
Epoch 3/50
313/313 [==============================] - 23s 74ms/step - loss: 3.8481 - accuracy: 0.1129 - top3_acc: 0.2442 - top_k_categorical_accuracy: 0.3357 - val_loss: 3.6380 - val_accuracy: 0.1741 - val_top3_acc: 0.3366 - val_top_k_categorical_accuracy: 0.4364
Epoch 4/50
313/313 [==============================] - 25s 79ms/step - loss: 3.7140 - accuracy: 0.1356 - top3_acc: 0.2862 - top_k_categorical_accuracy: 0.3799 - val_loss: 3.5351 - val_accuracy: 0.1895 - val_to

In [10]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [4.459047794342041, 4.0154523849487305, 3.8197011947631836, 3.6871085166931152, 3.5722954273223877, 3.47745418548584, 3.393366813659668, 3.302201271057129, 3.2290725708007812, 3.1396801471710205, 3.0733749866485596, 3.0040507316589355, 2.9434621334075928, 2.8845889568328857, 2.82080340385437, 2.7694625854492188, 2.7066643238067627, 2.65496826171875, 2.604931354522705, 2.549025535583496, 2.4959521293640137, 2.451040267944336, 2.39925479888916, 2.355045795440674, 2.301292896270752, 2.2622897624969482, 2.207249164581299, 2.159728765487671, 2.1127867698669434, 2.065460681915283, 2.019068956375122, 1.9761167764663696, 1.9300545454025269, 1.874473214149475, 1.8347859382629395, 1.78629469871521, 1.743043303489685, 1.6978930234909058, 1.6507058143615723, 1.6105663776397705, 1.5681939125061035, 1.5161161422729492, 1.471673846244812, 1.4251973628997803, 1.3736217021942139, 1.3472652435302734, 1.3027822971343994, 1.2465420961380005, 1.213927984237671, 1.1760339736938477], 'accuracy': [0.